In [3]:
# Notebook header <--- Always run this cell first!
import os, json
import ruamel.yaml as yaml
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from monty.json import MontyDecoder, MontyEncoder
from pymatgen.io.vasp.outputs import Outcar, Vasprun
from xml.etree.ElementTree import ParseError
from quotas.core import QuotasCalculator, WorkFunctionData, DielTensor

# Load the dictionary of all the structures for which we have calculated the required properties
with open("structure_dict.yaml", "r") as file:
    structure_dict = yaml.safe_load(file.read())

data_dir = "/mnt/data/mbercx/quotas"

# Process the QUOTAS data

## Parsing the VASP output files.

Calculating the secondary electron emission from the VASP output files is quite time consuming, as parsing the files is relatively slow. In order to quickly load the required data for calculating the SEE, it's better to store it as a QuotasCalculator JSON file for future use. The cell below defined some quick methods for processing the output files and writing a JSON file for each surface of each element. 

In [4]:
def load_and_check(vasprun_file):
    
    try:
        vr = Vasprun(vasprun_file)
    except ParseError:
        return False
    except FileNotFoundError:
        return False
    
    if not vr.converged or vr.dos_has_errors:
        return False
    
    return vr

def process_data(element, surfaces, functional="pbe"):
    
    optics_dir = os.path.join(data_dir, element, "bulk", functional + "_optics")
    
    if load_and_check(os.path.join(optics_dir, "vasprun.xml")):
    
        diel = DielTensor.from_file(os.path.join(optics_dir, "vasprun.xml"))
        out = Outcar(os.path.join(optics_dir, "OUTCAR"))
        out.read_freq_dielectric()
        plasma_frequencies = out.plasma_frequencies["intraband"] ** (1/2)
        diel.add_intraband_dieltensor(plasma_frequency=plasma_frequencies)
        
    else:
        return "optics_issue"
    
    surface_issues = []
    
    for surface in surfaces:
        
        dos_dir = os.path.join(data_dir, element, surface, functional + "_dos")
        dos_vr = load_and_check(os.path.join(dos_dir, "vasprun.xml"))
        
        if dos_vr:
            
            cdos = dos_vr.complete_dos
            workfunction_data = WorkFunctionData.from_dir(dos_dir)

            quotas = QuotasCalculator(
                cdos=cdos,
                workfunction_data=workfunction_data,
                dieltensor=diel,
                energy_spacing=0.05
            )
            quotas.to(os.path.join("data", element + "_" + surface + "_" + functional + ".json"))
        else:
            surface_issues.append("surface" + "_" + surface + "_issue")
            
    if len(surface_issues) > 0:
        return surface_issues
    else:
        return "A-Okay!"

The cell below loops over all elements and processes their VASP output files. It also checks if there are any issues with the output, and stores these issues under `issues.json`. Because running this cell can take a lot of time, there is a possibility that the connection to the jupyter server is interrupted. In order to make sure we don't start from scratch after such an interruption, the issues are written to its .json file after each element, and the cell ignores elements which are already in the `issues.json` file. If you want to run this cell again from scratch, you have to remove or rename the `issues.json` file.

In [5]:
try:
    with open("issues.json", "r") as file:
        issue_dict = json.loads(file.read())
except FileNotFoundError:
    issue_dict = {}

for element in structure_dict.keys():
    if element not in issue_dict.keys():
        issue_dict[element] = process_data(element, surfaces=structure_dict[element]["slabs"])
        with open("issues.json", "w") as file:
            file.write(json.dumps(issue_dict))

The cell below was used to rerun the processing script for specific elements, once the issues with their calculations had been fixed.

In [7]:
element = ""

with open("issues.json", "r") as file:
    issue_dict = json.loads(file.read())
    
issue_dict[element] = process_data(element, surfaces=structure_dict[element]["slabs"])

with open("issues.json", "w") as file:
    file.write(json.dumps(issue_dict))

KeyError: ''

In [8]:
with open("issues.json", "r") as file:
    issue_dict = json.loads(file.read())
issue_dict

{'Ag': 'A-Okay!',
 'Al': 'A-Okay!',
 'As': 'A-Okay!',
 'Au': 'A-Okay!',
 'Ba': 'A-Okay!',
 'Be': 'A-Okay!',
 'Bi': 'A-Okay!',
 'Ca': 'A-Okay!',
 'Cd': 'A-Okay!',
 'Co': 'A-Okay!',
 'Cr_afm': ['surface_100_issue', 'surface_110_issue'],
 'Cr_nm': 'A-Okay!',
 'Cs': 'A-Okay!',
 'Cu': 'A-Okay!',
 'Fe_alpha': 'A-Okay!',
 'Fe_gamma': 'A-Okay!',
 'Ga': 'A-Okay!',
 'Ge': 'A-Okay!',
 'Hf': 'A-Okay!',
 'Hg_alpha': 'A-Okay!',
 'Hg_beta': 'A-Okay!',
 'In': 'A-Okay!',
 'Ir': 'A-Okay!',
 'K': 'A-Okay!',
 'Li': 'A-Okay!',
 'Mg': 'A-Okay!',
 'Mo': 'A-Okay!',
 'Na': 'A-Okay!',
 'Nb': 'A-Okay!',
 'Ni': 'A-Okay!',
 'Os': 'A-Okay!',
 'Pb': 'A-Okay!',
 'Pd': 'A-Okay!',
 'Pt': 'A-Okay!',
 'Rb': 'A-Okay!',
 'Re': 'A-Okay!',
 'Rh': 'A-Okay!',
 'Ru': 'A-Okay!',
 'Sb': 'A-Okay!',
 'Sc': 'A-Okay!',
 'Si': 'A-Okay!',
 'Sn_alpha': 'A-Okay!',
 'Sn_beta': 'A-Okay!',
 'Sr': 'A-Okay!',
 'Ta': 'A-Okay!',
 'Tc': 'A-Okay!',
 'Ti': 'A-Okay!',
 'Tl': 'A-Okay!',
 'V': 'A-Okay!',
 'W': 'A-Okay!',
 'Y': ['surface_001_issue'],


## Construct a single dictionary with all of the yield results

Loading the `QuotasCalculator` .json files is already a lot faster than initializing them from the VASP output files, but if we want to develop some interactive tools for analyzing the data, it would be better to calculate the yield for all the ionization energies and store these results directly as a dictionary. First we set the ionization energies for the chosen ions and define a little script that calculates the yield for all ions with a specified set of `plasmon_parameters`.

In [9]:
# Define the effective ionization energies of the rare gas ions
ionization_energies = {
    "He": 24.59 - 2,
    "Ne": 21.56 - 1,
    "Ar": 15.76 - 0.5,
    "Kr": 12.00,
    "Xe": 10.13
}

def calculate_yield(quotas, plasmon_parameters=None):
    
    if plasmon_parameters is not None:
        quotas.set_up_plasmon_probabilities(
            bulk_parameter=plasmon_parameters["bulk"],
            surface_parameter=plasmon_parameters["surface"]
        )
        
    yield_dict = {}
    for ion in ionization_energies.keys():
        see = quotas.calculate_yield(ionization_energies[ion])
        yield_dict[ion] = {"energy": see["energy"], 
                           "yield": see["yield"], 
                           "total_yield": see["total_yield"]}
    return yield_dict

Next, we loop over all elements and gather all of their results into one dictionary of the following format:

```python
results_dict = {
    <element> = {
        "bulk": {"dieltensor": Dieltensor},
        <surface1>: {
            "complete_dos": CompleteDos,
            "workfunction_data": WorkfunctionData,
            "yield_results": {
                "energy": see["energy"], 
                "yield": see["yield"], 
                "total_yield": see["total_yield"]
            }
        }
        <surface2>: {
            ...
        }
        ...
    }
    ...
}
```

This takes quite a bit of time, and results in a rather large `.json` file that contains _all_ results for the bulk and surfaces of all elements. Note that the cell must complete without being interrupted, as the `results.json` file is only written at the end. This is because writing the file at every iteration slows down the process significantly. 

In [10]:
results_dict = {element: {} for element in structure_dict.keys()}

with open("issues.json", "r") as file:
    issue_dict = json.loads(file.read())
    
for element in structure_dict.keys():
    
    print(element)
    
    for surface in [surface for surface in structure_dict[element]["slabs"].keys() 
         if not 'surface_' + surface + '_issue' in issue_dict[element]]:

        quotas = QuotasCalculator.from_file("data/" + element + "_" + surface + "_pbe.json")
        
        if "bulk" not in results_dict[element].keys():
            results_dict[element]["bulk"] = {"dieltensor": quotas.dieltensor}
        
        results_dict[element][surface] = {
            "complete_dos": quotas.cdos,
            "workfunction_data": quotas.workfunction_data,
            "yield_results": calculate_yield(quotas)
        }
        
with open("results.json", "w") as file:
    file.write(json.dumps(results_dict, cls=MontyEncoder))

Ag
Al
As
Au


/home/mbercx/python/quotas/quotas/core.py:602: RuntimeWarning: invalid value encountered in double_scalars
  scatter_distribution, self.energies)


Ba
Be
Bi
Ca
Cd
Co
Cr_afm
Cr_nm
Cs
Cu
Fe_alpha
Fe_gamma
Ga
Ge
Hf
Hg_alpha
Hg_beta
In
Ir
K
Li
Mg
Mo
Na
Nb
Ni
Os
Pb
Pd
Pt
Rb
Re
Rh
Ru
Sb
Sc
Si
Sn_alpha
Sn_beta
Sr
Ta
Tc
Ti
Tl
V
W
Y
Zn
Zr


## Lightweight dictionary with all workfunctions/occupied band widths/yields.

The `results.json` file generated above is rather massive, and contains way more data than is necessary for analyzing trends in the total yield / work function. Hence, we will also generate a smaller set of `.json` file that only contain the final results.

In [11]:
yield_dict = {element: {} for element in structure_dict.keys()}
yield_np_dict = {element: {} for element in structure_dict.keys()}

with open("issues.json", "r") as file:
    issue_dict = json.loads(file.read())
    
def get_participating_band_width(cdos, ionization_energy):
    
    dos = cdos.get_densities()
    dos[cdos.energies < cdos.efermi - ionization_energy] = 0
    dos[cdos.energies > cdos.efermi] = 0
    energy_range = cdos.energies[np.nonzero(dos)]
    return energy_range[-1] - energy_range[0]
    
for element in structure_dict.keys():
    print(element)
    
    for surface in structure_dict[element]["slabs"].keys():
        if not 'surface_' + surface + '_issue' in issue_dict[element]:
            
            quotas = QuotasCalculator.from_file("data/" + element + "_" + surface + "_pbe.json")
            yield_dict[element][surface] = calculate_yield(quotas)
            yield_np_dict[element][surface] = calculate_yield(
                quotas, plasmon_parameters={"bulk": 0, "surface": 0}
            )
            yield_dict[element][surface]["work_function"] = quotas.workfunction_data.work_function
            yield_np_dict[element][surface]["work_function"] = quotas.workfunction_data.work_function
            
with open("yield.json", "w") as file:
    file.write(json.dumps(yield_dict, cls=MontyEncoder))
with open("yield_np.json", "w") as file:
    file.write(json.dumps(yield_np_dict, cls=MontyEncoder))

Ag
Al
As
Au
Ba
Be
Bi
Ca
Cd
Co
Cr_afm
Cr_nm
Cs
Cu
Fe_alpha
Fe_gamma
Ga
Ge
Hf
Hg_alpha
Hg_beta
In
Ir
K
Li
Mg
Mo
Na
Nb
Ni
Os
Pb
Pd
Pt
Rb
Re
Rh
Ru
Sb
Sc
Si
Sn_alpha
Sn_beta
Sr
Ta
Tc
Ti
Tl
V
W
Y
Zn
Zr


In [ ]:
with open("results.json", "r") as file:
    results_dict = json.loads(file.read(), cls=MontyDecoder)
    
with open("yield.json", "r") as file:
    yield_dict = json.loads(file.read(), cls=MontyDecoder)

with open("yield_np.json", "r") as file:
    yield_np_dict = json.loads(file.read(), cls=MontyDecoder)
    
for element in yield_dict.keys():
    for surface in yield_dict[element].keys():
        for ion in ionization_energies.keys():
            band_width = get_participating_band_width(
                results_dict[element][surface]["complete_dos"], ionization_energies[ion]
            )
            yield_dict[element][surface][ion].update({"band_width": band_width})
            yield_np_dict[element][surface][ion].update({"band_width": band_width})

with open("yield.json", "w") as file:
    file.write(json.dumps(yield_dict, cls=MontyEncoder))
with open("yield_np.json", "w") as file:
    file.write(json.dumps(yield_np_dict, cls=MontyEncoder))